In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
import sys
sys.path.insert(0, "../../esinet")
sys.path.insert(0, "../")

import numpy as np
import mne
from esinet.forward import create_forward_model, get_info
from esinet import Simulation, Net
from esinet.util import unpack_fwd
pp = dict(hemi='both', surface='white')

In [2]:
info = get_info("standard_1020")
fwd = create_forward_model(info=info)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


In [9]:
from laura import *

settings=dict(duration_of_trial=0.1, target_snr=999)
sim_test = Simulation(fwd, info, settings=settings).simulate(2)
evoked = sim_test.eeg_data[0].average()
stc = apply_laura(evoked, fwd, reg=10)

sim_test.source_data[0].plot(**pp)
stc.plot(**pp)

stc = apply_laura(evoked, fwd, reg=10, drop_off=1.5)
stc.plot(**pp)
stc = apply_laura(evoked, fwd, reg=10, drop_off=2.5)
stc.plot(**pp)


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 2003.01it/s]


source data shape:  (1284, 100) (1284, 100)


100%|██████████| 2/2 [00:00<00:00, 38.19it/s]
c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:159: RuntimeWarning: divide by zero encountered in power
  W_j = np.matrix(inv(np.matmul(M.T, M)))


Using control points [0.00000000e+00 0.00000000e+00 1.22852105e-09]
Using control points [7.08878605e-11 9.67046989e-11 4.10158229e-10]


...

Using control points [9.40647613e-11 1.29199477e-10 5.41616144e-10]


c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:159: RuntimeWarning: divide by zero encountered in power
  W_j = np.matrix(inv(np.matmul(M.T, M)))


Using control points [4.06013663e-11 5.80337218e-11 2.31381293e-10]


  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [7.75620404e-11 1.04579567e-10 2.96406879e-10]
Using control points [4.93739103e-11 6.80309157e-11 1.48713476e-10]
Using control points [0.00000000e+00 0.00000000e+00 8.33102631e-10]
Using control points [7.72754320e-11 1.06522181e-10 2.32890228e-10]
Using control points [1.21437576e-10 1.63756047e-10 4.64151147e-10]


In [4]:
from laura import *
from scipy.spatial.distance import cosine

settings=dict(duration_of_trial=0.1, target_snr=5)
sim_test = Simulation(fwd, info, settings=settings).simulate(10)
regs = [0,0.001,0.01, 0.1, 1, 10, 100]
errors = []
for reg in regs:
    print("REG: ", reg)
    stc_data = np.stack([apply_laura(epochs.average(), fwd, reg=reg).data for epochs in sim_test.eeg_data], axis=0).flatten()
    true_data = np.stack([source.data for source in sim_test.source_data], axis=0).flatten()
    error = cosine(stc_data, true_data) 
    # error = ((stc_data-true_data)**2).mean()
    errors.append(error)

Simulating data based on sparse patches.


100%|██████████| 10/10 [00:00<00:00, 5012.31it/s]


source data shape:  (1284, 100) (1284, 100)


100%|██████████| 10/10 [00:00<00:00, 57.96it/s]


REG:  0
-- number of adjacent vertices : 1284


c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:153: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
REG:  0.001
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
REG:  0.01
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent vertices : 1284
-- number of adjacent verti

In [6]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.figure()
plt.plot(regs, errors)
plt.xlabel("regs")
plt.ylabel("error")
# plt.ylim(3e-19, 1.01e-18)

Text(0, 0.5, 'error')

# Amendment: n_neighbors

In [ ]:
settings=dict(duration_of_trial=0.1, target_snr=999, number_of_sources=2, extents=(25,35))
sim_test = Simulation(fwd, info, settings=settings).simulate(10)

evoked = sim_test.eeg_data[0].average().data
sim_test.source_data[0].plot(**pp)

In [ ]:
stc.plot(**pp)

In [ ]:
settings=dict(duration_of_trial=0.1, target_snr=9999)
sim_test = Simulation(fwd, info, settings=settings).simulate(2)
x = sim_test.eeg_data[0].average().data
reg = 0.01
G = inv(M.T*M) * leadfield.T * inv( leadfield * inv(M.T * M) * leadfield.T + reg * np.identity(leadfield.shape[0]) )
y_hat = G*x * 1e-10
stc = mne.SourceEstimate(y_hat, vertices, tmin=evoked.tmin, tstep=1/evoked.info["sfreq"], subject=forward["src"]._subject)
stc.plot(**pp)